# 3. 신경망
* * *

+ 퍼셉트론으로 복잡한 함수도 표현할 수 있음
+ 그러나, 가중치를 설정하는 작업 ( 원하는 결과를 출력하도록 가중치 값을 적절히 정하는 작업 ) 은 여전히 사람이 수동으로 해야함
</br></br>
+ 신경망의 중요한 성질 == 가중치 매개변수의 적절한 값을 데이터로부터 잗ㅇ으로 학습하는 능력

## 3.6 손글씨 숫자 인식
+ 이미 학습된 매개변수를 사용하여 학습 과정은 생략하고, 추론 과정만 구현
+ 추론과정 == 신경망의 순전파 ( forward propagation )
</br></br>
+ 신경망은 두 단계를 거쳐 문제를 해결함
    + 훈련 데이터 ( 학습 데이터 ) 를 사용해 가중치 매개변수 학습
    + 추론 단계에서 앞서 학습한 매개변수를 사용하여 입력 데이터를 분류

##### <br><br><span style="color: mediumaquamarine;">**MNIST 데이터셋**</spen>
+ <spen style="color: gold;">**MNIST 데이터셋**</spen> == 손글씨 숫자 이미지 집합 ( 0 ~ 9 )
    + 기계학습 분야에서 아주 유명한 데이터셋
    + 간단한 실험부터 논문으로 발표되는 연구까지 다양한 곳에서 이용함
    + 이미지 인식이나 기계학습 논문들을 읽다 보면 실험용 데이터로 자주 등장하는 것을 확인할 수 있음
+ 일반적으로 훈련 이미지들을 사용하여 모델을 학습하고, 학습한 모델로 시험 이미지들을 얼마나 정확하게 분류하는지를 평가함
</br></br>
+ 28 X 28 크기의 회색조 이미지 ( 채널 1 )
+ 각 픽셀은 0 ~ 255까지의 값을 취함
+ 가 이미지가 실제 의미하는 숫자가 레이블로 붙어있음

In [1]:
import sys, os
sys.path.append(os.pardir) # 부모 디렉터리의 파일을 가져올 수 있도록 설정
from dataset.mnist import load_mnist

# load_mnist가 MNIST 데이터를 받아와야 하니 최초 실행 시에는 인터넷에 연결된 상태여야 한다.
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False)

# 각 데이터의 형상 출력
print(x_train.shape)
print(t_train.shape)
print(x_test.shape)
print(t_test.shape)

(60000, 784)
(60000,)
(10000, 784)
(10000,)


+ ```load_mnist``` 함수 == 읽은 MNIST 데이터를 "( 훈련 이미지, 훈련 레이블 ), ( 시험 이미지, 시험 레이블 )" 형식으로 반환
    <spen style="color: rosybrown;">**인수**</spen>
    + ```normalize``` == 입력 이미지의 픽셀 값을 0.0 ~ 1.0 사이의 값으로 정규화할지를 정함
        + False == 입력 이미지의 픽셀은 원래 값 그대로 0 ~ 255 사이의 값을 유지
    + ```flatten``` == 입력 이미지를 평탄하게, 즉 1차원 배열로 만들지를 정함
        + False == 입력 이미지가 1 X 28 X 28의 3차원 배열
        + True == 입력 이미지가 784개의 원소로 이뤄진 1차원 배열
    + ```one_hot_lable``` == 원-핫 인코딩 형태로 저장할지를 정함
        + False == '7', '2'와 같이 숫자 형태의 레이블을 저장
        + True == 레이블을 원-핫 인코딩하여 저장 ( <spen style="color: gold;">**원-핫 인코딩 ( one-hot encoding )**</spen> == 정답을 뜻하는 원소만 1이고 ( hot하고 ) 나머지는 모두 0인 배열 )